In [7]:
model = "qwen/qwen3-30b-a3b"
tailscale_server = "https://desktop-3oeimac.tail3b962f.ts.net"
chat_completion_api = tailscale_server + "/api/v0/chat/completions"
embedding_model = "text-embedding-qwen3-embedding-8b@q5_0"

In [8]:
transcr_location = "./transcriptions/The_Secrets_of_bodyweight_only_parsed_text.txt"

with open(transcr_location) as file:
    f = file.read()
    print(f)

 How is bodyweight strength training different for adults compared to children? Good question. First seminar I ever did for adults. I'm a national team coach. And what I learned is it really skewed my expectations of what was normal because that was my world. That was who I was surrounded by. They weren't exceptional. They just were. These are the expectations. I ask them to do things and they can perform at this level. So I run an event for adults, first GB seminar on their schedule for the day. And I'm going to take them through just a gentle warmup, nothing hard, nothing demanding. I've got 15 minutes on the schedule, 10 minutes, 15 if we're dragging. I can't get them to focus. They're not paying attention, whatever. And it took an hour and a half for a nothing warmup. They were wrecked from it. They were wrecked. They were exhausted, sweat everywhere, body strewn all over the floor. I remember turning to my staff and I go, you know, what do I do now? You know, they failed warmup. Y

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=3200, chunk_overlap=40)

docs = text_splitter.split_text(f)

In [10]:
len(docs)

6

In [11]:
print(docs[0])

 How is bodyweight strength training different for adults compared to children? Good question. First seminar I ever did for adults. I'm a national team coach. And what I learned is it really skewed my expectations of what was normal because that was my world. That was who I was surrounded by. They weren't exceptional. They just were. These are the expectations. I ask them to do things and they can perform at this level. So I run an event for adults, first GB seminar on their schedule for the day. And I'm going to take them through just a gentle warmup, nothing hard, nothing demanding. I've got 15 minutes on the schedule, 10 minutes, 15 if we're dragging. I can't get them to focus. They're not paying attention, whatever. And it took an hour and a half for a nothing warmup. They were wrecked from it. They were wrecked. They were exhausted, sweat everywhere, body strewn all over the floor. I remember turning to my staff and I go, you know, what do I do now? You know, they failed warmup. Y

In [12]:
def format_reasoning_model_prompt(prompt:str,no_think=True):
    if no_think:
        return prompt + " /no_think"
    return prompt

In [13]:
import json
with open("./prompt_catalog.json") as catalog_file:
    json_data = json.load(catalog_file)
    
print(json_data.get('system'))

{'transcript_summarizer': 'You are part of a system whose focus is giving practical strategies for increasing performance in workouts relating to bodyweight and gymnastics. \n <ROLE> \n Your primary role is to assist users by summing up transcripts videos given by the user as input, focusing on finding practices which have been useful in workout plans. Summing up means that the generated summary should not be longer of the provided input. Therefore try being more concise, not too verbose and straight to the point. \n </ROLE>'}


In [93]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage


# Initialize the LLM pointing to your LM Studio server
llm = ChatOpenAI(
    base_url=f"{tailscale_server}/v1",  # LM Studio server URL
    api_key="lm-studio",  # Can be any string for local models
    model=model,  # Can be any string for local models
)


messages = [
    SystemMessage(json_data.get('system').get('transcript_summarizer')),
    HumanMessage(format_reasoning_model_prompt(docs[0])),
]

response = llm.invoke(messages)


print(response.content)

<think>

</think>

Key differences in bodyweight strength training for adults vs. children:

- **Expectations & Capacity:** Adults often have lower baseline fitness, mobility, and coordination compared to trained children (e.g., 15-minute warmup became an hour-long ordeal).
- **Warmup Focus:** For adults, basic mobility and joint preparation are critical; for kids, advanced movements like rope climbs or complex pushups may be part of the warmup.
- **Training Intensity:** Adults need more structured, progressive prep before skill work. Kids often train at higher intensity from the start.
- **Mindset & Work Ethic:** Adults may lack the consistent effort seen in children who are eager to learn and improve.
- **Physical Adaptation:** Children naturally adapt faster due to flexibility and growth; adults require more deliberate strength and mobility work for similar gains.


In [94]:
messages = [
    SystemMessage(json_data.get('system').get('transcript_summarizer')),
    HumanMessage(format_reasoning_model_prompt(docs[0])),
    ]
    
responses = []
for doc in docs:
    messages[1] = HumanMessage(format_reasoning_model_prompt(doc))
    response = llm.invoke(messages)

    print(response.content)
    responses.append(response.content)

<think>

</think>

Bodyweight strength training for adults differs from children in focus, intensity, and expectations. Adults often lack foundational mobility, stability, and coordination, requiring more time on warmups and basic movements. Children, especially elite athletes, are naturally more adaptable and can handle higher-intensity, complex routines with less preparation. For adults, the emphasis is on building a strong base through controlled, progressive work, while children can push into advanced skills sooner. Consistency and effort matter more than innate talent for adults, as progress often comes from dedicated, structured training over time.
<think>

</think>

This transcript discusses key principles for effective bodyweight and gymnastics training, emphasizing:

- **Progression over overload**: Starting with controlled movements (e.g., chin-over-bar pull-ups) before progressing to kipping or weighted work.
- **Mobility vs. flexibility**: Mobility is about strength through

In [101]:
import re

def remove_think_tags(text):
    """
    Remove <think>...</think> blocks from text using regex.
    Handles multiline content and whitespace.
    """
    # Pattern explanation:
    # <think>     - matches opening tag
    # .*?         - matches any characters (non-greedy)
    # </think>    - matches closing tag
    # re.DOTALL   - makes . match newlines too
    pattern = r'<think>.*?</think>'
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text.strip()

In [103]:
idx = 1
cleaned_docs = []
with open("summary_transcript.txt", "w") as output_file:
    for response in responses:
        cleaned_response = remove_think_tags(response)
        cleaned_docs.append(cleaned_response)
        output_file.write(f"PART {idx}: \n\n" )
        output_file.write(f"{cleaned_response}\n\n")
        idx += 1

In [6]:
!pip install langchain langchain-openai
!pip install sentence-transformers
!pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.4 MB/s eta 0:00:00m eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.7/367.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 16.7 MB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 15.0 MB/s eta 0:00:0031m14.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 17.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.5/352.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [139]:
# from langchain.embeddings.base import Embeddings
from chromadb import Documents, EmbeddingFunction, Embeddings
import requests
import logging
from typing import List, Optional

class CustomEmbeddingModel(Embeddings):
    def __init__(self, 
                 model_name: str,
                 base_url: str = f"{tailscale_server}/v1",
                 api_key: str = "lm-studio",
                 timeout: int = 30):
        self.model_name = model_name
        self.base_url = base_url.rstrip('/')
        self.api_key = api_key
        self.timeout = timeout
        print(self.base_url)
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        # Test connection
        self._test_connection()
    
    def _test_connection(self):
        """Test if LM Studio is accessible."""
        try:
            response = requests.get(f"{self.base_url}/models", 
                                  headers=self.headers, 
                                  timeout=5)
            response.raise_for_status()
            logging.info("Successfully connected to LM Studio")
        except Exception as e:
            logging.warning(f"Could not connect to LM Studio: {e}")
    
    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        url = f"{self.base_url}/embeddings"
        
        payload = {
            "model": self.model_name,
            "input": texts
        }
        
        try:
            print(payload)
            response = requests.post(url, 
                                   headers=self.headers, 
                                   json=payload,
                                   timeout=self.timeout)
            response.raise_for_status()
            
            data = response.json()
            return [item["embedding"] for item in data["data"]]
            
        except requests.exceptions.RequestException as e:
            raise Exception(f"LM Studio API error: {e}")
        except KeyError as e:
            raise Exception(f"Unexpected API response format: {e}")
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # Process in batches if needed
        batch_size = 10  # Adjust based on your needs
        all_embeddings = []
        
        all_embeddings = self._get_text_embeddings(texts)
        # for i in range(0, len(texts), batch_size):
        #     batch = texts[i:i + batch_size]
        #     batch_embeddings = self._get_embeddings(batch)
        #     all_embeddings.extend(batch_embeddings)
        
        return all_embeddings
    
    def embed_query(self, text: str) -> List[float]:
        embeddings = self._get_text_embeddings([text])
        return embeddings[0]
    
    
class CustomEmbeddingFunction(EmbeddingFunction):
    
    def __init__(self, embedding_model: CustomEmbeddingModel):
        super().__init__()
        self.embedding_model = embedding_model
        
    def __call__(self, input: list[str]) -> Embeddings:
        # embed the documents somehow
        embeddings_list = []
        for doc in input:
            embeddings_list.append(self.embedding_model.embed_query(doc))
        return embeddings_list

In [114]:
embedder_model = CustomEmbeddingModel("text-embedding-nomic-embed-text-v1.5")
embedded_text = embedder_model.embed_query(responses[0])


https://desktop-3oeimac.tail3b962f.ts.net/v1
{'model': 'text-embedding-nomic-embed-text-v1.5', 'input': ['<think>\n\n</think>\n\nBodyweight strength training for adults differs from children in focus, intensity, and expectations. Adults often lack foundational mobility, stability, and coordination, requiring more time on warmups and basic movements. Children, especially elite athletes, are naturally more adaptable and can handle higher-intensity, complex routines with less preparation. For adults, the emphasis is on building a strong base through controlled, progressive work, while children can push into advanced skills sooner. Consistency and effort matter more than innate talent for adults, as progress often comes from dedicated, structured training over time.']}


In [56]:
import numpy as np

embedded_array = np.array(embedded_text)
embedded_array

array([-2.40165889e-02,  9.26086456e-02, -1.77638948e-01,  2.21012011e-02,
       -1.68802105e-02,  6.11814186e-02,  6.50262088e-02, -1.39255112e-03,
        5.06963544e-02, -2.18267180e-03, -1.52600724e-02, -4.32971828e-02,
        6.14499450e-02, -1.45928878e-02,  7.02236444e-02, -1.43646458e-02,
        2.70669851e-02, -1.16764188e-01,  2.11075768e-02,  2.04808470e-02,
       -3.82633060e-02, -4.16273139e-02, -1.00941630e-02,  6.19568489e-03,
       -1.27001451e-02,  6.86157942e-02, -1.28094889e-02, -3.29600349e-02,
       -3.02738640e-02,  1.91103928e-02,  2.79848333e-02, -3.33337672e-02,
        4.13336568e-02, -4.88125347e-02, -4.07791045e-03,  2.78606508e-02,
        9.03669149e-02,  5.27128316e-02,  1.51131814e-02, -4.41161506e-02,
        1.75218042e-02,  8.59320723e-03, -1.10567929e-02, -5.89739569e-02,
       -7.38264667e-03, -2.78790537e-02,  1.29105551e-02, -5.46865128e-02,
        3.15566771e-02,  4.65718023e-02, -4.61992286e-02, -4.84758429e-02,
        9.76716727e-03, -

In [115]:
import hashlib

def create_id_from_string(input_string):
    # Use SHA-256 for strong hashing
    hash_object = hashlib.sha256(input_string.encode())
    # Get the hex digest (a string representation)
    return hash_object.hexdigest()

my_string = docs[0]
unique_id = create_id_from_string(my_string)


In [140]:
import chromadb

client = chromadb.Client()
collection = client.get_or_create_collection(name="transcript_collections", embedding_function= CustomEmbeddingFunction(embedder_model))


/tmp/ipykernel_756795/3793750012.py:82: DeprecationWarning: The class CustomEmbeddingFunction does not implement __init__. This will be required in a future version.
  super().__init__()


In [141]:
ids = [create_id_from_string(doc) for doc in cleaned_docs]
documents = cleaned_docs
metadatas = [{"title": "The_Secrets_of_bodyweight"} for _ in range(len(cleaned_docs))]
assert len(ids) == len(documents) == len(metadatas)
# #todo: the metadata should be enriched with the youtube video title
collection.add(ids=[create_id_from_string(doc) for doc in cleaned_docs], documents=cleaned_docs, metadatas=[{"title": "The_Secrets_of_bodyweight"} for i in range(len(cleaned_docs))] )

{'model': 'text-embedding-nomic-embed-text-v1.5', 'input': ['Bodyweight strength training for adults differs from children in focus, intensity, and expectations. Adults often lack foundational mobility, stability, and coordination, requiring more time on warmups and basic movements. Children, especially elite athletes, are naturally more adaptable and can handle higher-intensity, complex routines with less preparation. For adults, the emphasis is on building a strong base through controlled, progressive work, while children can push into advanced skills sooner. Consistency and effort matter more than innate talent for adults, as progress often comes from dedicated, structured training over time.']}
{'model': 'text-embedding-nomic-embed-text-v1.5', 'input': ['This transcript discusses key principles for effective bodyweight and gymnastics training, emphasizing:\n\n- **Progression over overload**: Starting with controlled movements (e.g., chin-over-bar pull-ups) before progressing to kip

In [142]:
collection.count()

6

In [148]:
question = "what are the suggestion to train gymnastics for an adult?"
collection.get(include=["documents"]) # Only 'ids' and 'documents' are returned

results = collection.query(
    query_texts=[question], # Chroma will embed this for you
    n_results=2 # how many results to return
)


{'model': 'text-embedding-nomic-embed-text-v1.5', 'input': ['what are the suggestion to train gymnastics for an adult?']}


In [150]:
print(results.get('documents'))

[['Bodyweight strength training for adults differs from children in focus, intensity, and expectations. Adults often lack foundational mobility, stability, and coordination, requiring more time on warmups and basic movements. Children, especially elite athletes, are naturally more adaptable and can handle higher-intensity, complex routines with less preparation. For adults, the emphasis is on building a strong base through controlled, progressive work, while children can push into advanced skills sooner. Consistency and effort matter more than innate talent for adults, as progress often comes from dedicated, structured training over time.', 'This transcript discusses key principles for effective bodyweight and gymnastics training, emphasizing:\n\n- **Progression over overload**: Starting with controlled movements (e.g., chin-over-bar pull-ups) before progressing to kipping or weighted work.\n- **Mobility vs. flexibility**: Mobility is about strength through range of motion, not just fl

In [146]:
from transformers import AutoTokenizer, AutoModel

# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

/home/gabriele.local/Documents/repos/caption_videos/venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [155]:
import torch
import time

start = time.time()
scores = []

# Function to compute MaxSim
def maxsim(query_embedding, document_embedding):
    # Expand dimensions for broadcasting
    # Query: [batch_size, query_length, embedding_size] -> [batch_size, query_length, 1, embedding_size]
    # Document: [batch_size, doc_length, embedding_size] -> [batch_size, 1, doc_length, embedding_size]
    expanded_query = query_embedding.unsqueeze(2)
    expanded_doc = document_embedding.unsqueeze(1)

    # Compute cosine similarity across the embedding dimension
    sim_matrix = torch.nn.functional.cosine_similarity(expanded_query, expanded_doc, dim=-1)

    # Take the maximum similarity for each query token (across all document tokens)
    # sim_matrix shape: [batch_size, query_length, doc_length]
    max_sim_scores, _ = torch.max(sim_matrix, dim=2)

    # Average these maximum scores across all query tokens
    avg_max_sim = torch.mean(max_sim_scores, dim=1)
    return avg_max_sim

# Encode the query
query_encoding = tokenizer(question, return_tensors='pt')
query_embedding = model(**query_encoding).last_hidden_state.mean(dim=1)

# Get score for each document
for document in results.get('documents'):
    document_encoding = tokenizer(document[0], return_tensors='pt', truncation=True, max_length=512)
    document_embedding = model(**document_encoding).last_hidden_state

    # Calculate MaxSim score
    score = maxsim(query_embedding.unsqueeze(0), document_embedding)
    scores.append({
        "score": score.item(),
        "document": document[0],
    })

print(f"Took {time.time() - start} seconds to re-rank documents with ColBERT.")

Took 0.31059885025024414 seconds to re-rank documents with ColBERT.


In [156]:
# Sort the scores by highest to lowest and print
sorted_data = sorted(scores, key=lambda x: x['score'], reverse=True)
print(json.dumps(sorted_data, indent=2))

[
  {
    "score": 0.7187464833259583,
    "document": "Bodyweight strength training for adults differs from children in focus, intensity, and expectations. Adults often lack foundational mobility, stability, and coordination, requiring more time on warmups and basic movements. Children, especially elite athletes, are naturally more adaptable and can handle higher-intensity, complex routines with less preparation. For adults, the emphasis is on building a strong base through controlled, progressive work, while children can push into advanced skills sooner. Consistency and effort matter more than innate talent for adults, as progress often comes from dedicated, structured training over time."
  }
]


In [3]:
# step 1: we look for existing transcript for the selected videos.
import glob 
from langchain_community.document_loaders import YoutubeLoader
from pytube import YouTube


media_dir = "./media/it"
loaded_videos = []
for doc in glob.glob(pathname=f"{media_dir}/*.txt", recursive=True):
    with open(doc, "r") as file:
        lines = file.readlines()
        
        
        for line in lines:
            url = line.strip()
            yt = YouTube(url)
            try:
                title = yt.title
            except Exception:
                title = None
                print(f'title not found for {url}')

            try:  
                loader = YoutubeLoader.from_youtube_url(
                    url, language=['en', 'it'], continue_on_failure=True
                )
            
                transcript = loader.load()
            except Exception:
                print(f'Error loading the transcript for {url}')
                transcript = None
            loaded_videos.append({"url": url, "title": title, "transcript": transcript })
            


print(loaded_videos)

title not found for https://www.youtube.com/watch?v=NyjXMMBPvSA
Error loading the transcript for https://www.youtube.com/watch?v=NyjXMMBPvSA
title not found for https://www.youtube.com/watch?v=mQENVePdT5A&t=6s
Error loading the transcript for https://www.youtube.com/watch?v=mQENVePdT5A&t=6s
[{'url': 'https://www.youtube.com/watch?v=NyjXMMBPvSA', 'title': None, 'transcript': None}, {'url': 'https://www.youtube.com/watch?v=mQENVePdT5A&t=6s', 'title': None, 'transcript': None}]


In [ ]:
#step 2: we load audio and convert them if no transcript is available yet.
from langchain_community.document_loaders import YoutubeAudioLoader
from langchain.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers.audio import OpenAIWhisperParserLocal
from langchain_community.document_loaders.blob_loaders.file_system import FileSystemBlobLoader
from typing import AsyncIterator, Iterator

import whisper

# model = whisper.load_model("turbo")
# result = model.transcribe("./test/🔥COME MIGLIORARE TUTTE LE HALF LAY？🔥.m4a")
# print(result["text"])

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class CustomWhisperLoader(BaseLoader):
    """An example document loader that reads a file line by line."""

    def __init__(self, save_dir: str, sel_model = "turbo") -> None:
        """Initialize the loader with a file path.

        Args:
            file_path: The path to the file to load.
        """
        self.save_dir = save_dir
        self.model = whisper.load_model(sel_model)

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, you should use a generator
        to yield documents one by one.
        """
        print('dddf',self.save_dir)

    def lazy_parse(self, file) -> Iterator[Document]:  # <-- Does not take any arguments
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, you should use a generator
        to yield documents one by one.
        """
        print('dd',file.path)
        result =self.model.transcribe(file.path)
        result["text"]

urls_to_process = []
save_dir = './test'
for v in loaded_videos:
    
    if v.get('transcript',None) is None:
        urls_to_process.append(v.get('url', None))
        

loader = GenericLoader(
        YoutubeAudioLoader(
            urls_to_process, 
            save_dir
        ),
        CustomWhisperLoader(save_dir))
loader.load()
    # print(documents)

In [ ]:
!pip install langchain-community
%pip install --upgrade --quiet  youtube-transcript-api
!pip install yt_dlp
!pip install librosa
!pip install pydub
!pip install pytube
!pip install youtube-transcript-api


[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [179]:
len(documents)
documents[0]

Document(metadata={'source': 'test/Pedro Pascal Monologue - SNL.m4a'}, page_content=" Ladies and gentlemen Pedro Pascal Thank you. Thank you. Thank you very much. I'm so excited to be here. I uh, I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be here. I'm so excited to be 

In [192]:
!pip install youtube-transcript-api
!pip install pytube


[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [197]:
try:
    import pytube
    import youtube_transcript_api
    print("Both libraries are installed and importable.")
except ImportError as e:
    print("Import error:", e)


Both libraries are installed and importable.


In [205]:
from pytube import YouTube

yt = YouTube("https://www.youtube.com/watch?v=NyjXMMBPvSA")
print(yt.thumbnail_url)

HTTPError: HTTP Error 400: Bad Request

In [234]:
loaded_videos[0][0]

Document(metadata={'source': 'NyjXMMBPvSA'}, page_content="[Musica] Ciao ragazzi, benvenuti sul mio canale YouTube e sulla mia pagina Facebook. Oggi sono qua per parlarvi delle trazioni e in particolare vi voglio parlare di quello che secondo me è il problema più grosso delle trazioni che ho riscontrato finora. Questo perché la trazione è un movimento che parte da completa estensione degli atti superiori e poi prevede di portare il mento sopra. Poi eh le regole moderne dello street lifting che a me personalmente fanno cagare, prevede che si vada in questa posizione qua e sia sufficiente fare così. Purtroppo questo, secondo me, non è un movimento e un'esecuzione che riflette una persona forte. È più probabile, secondo me, che rifletta una persona che è molto esplosiva, cioè che riesce ad accelerare molto, perché la qualità, la capacità di grindare viene solo se voi andate col mento sopra e tenete la posizione. Questo è il mio punto di vista. Poi c'è la final rap. Le regole le decidono l

In [250]:
!pip install -U openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 11.7 MB/s eta 0:00:00 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached more_itertools-10.7.0-py3-none-any.whl (65 kB)
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=f2d5dd4df7b722d28808d10fe5843e3ef0fc2386bf9ebe77d87a4b6581f7bf5f
  Stored in directory: /home/gabriele.local/.cache/pip/wheels/be/22/86/dcf050d50e02b3bb67b5a3b289be2f3376ce7f8717d012d6a8
Successfully built openai-whisper

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [16]:
urls_to_process

['https://www.youtube.com/watch?v=NyjXMMBPvSA']